In [11]:
import numpy as np
import scipy.fftpack as fourier
import matplotlib
import matplotlib.pyplot as plt
import scipy.io.wavfile as waves
import pyaudio as pa 
import struct 
import statsmodels.api as sm
from scipy.signal import find_peaks